In [1]:
import cv2
import sys
from PyQt5.QtCore import Qt,QThread, pyqtSignal, pyqtSlot
from PyQt5.QtWidgets import QApplication, QMainWindow, QFileDialog
from PyQt5.QtGui import QPixmap
from PyQt5 import uic
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.cluster import KMeans

In [2]:
min_ratio, max_ratio = 0.2, 2

class RL():
    def __init__(self):
        self.df = pd.DataFrame(columns=['Hue', 'Saturation', 'label'])
        self.X_train = None
        self.kmeans = KMeans(n_clusters=3, max_iter=5000, n_init='auto', random_state=511)
        self.markers, self.colors = ['^', 's', '*'], ['r', 'g', 'b']
        self.radius = 50#100
        self.RED = 0b00000001
        self.GREEN = 0b00000010
        self.BLUE = 0b00000011
        self.NOCLUE = 0b00000000
        self.rl, self.rr = 0.2, 2
        
        self.red_class, self.green_class, self.blue_class = 1, 2, 3
    
    def find_circle(self, filename=None, img = None):
        if img is None:
            #print(filename)
            src = cv2.imread(filename)
        else:
            src = img
        print(src.shape)
        src = cv2.GaussianBlur(src, (5,5), 0)
        src = cv2.resize(src, (0,0), fx=0.3, fy=0.3)
        src_rgb = cv2.cvtColor(src, cv2.COLOR_BGR2RGB)
        src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
        src_hsv = cv2.cvtColor(src, cv2.COLOR_BGR2HSV)
        dst = src_rgb.copy()

        circles = cv2.HoughCircles(src_gray, cv2.HOUGH_GRADIENT, 1, 100, param1=250, param2=20, minRadius=int(self.radius*min_ratio), maxRadius=int(self.radius*max_ratio))
        
        amh, ams = [], []
        if circles is None:
            return None, (None, None), None
        
        for i in circles[0]:
            cv2.circle(dst, (int(i[0]), int(i[1])), int(i[2]), (255,255,255), 2)
            pt1, pt2 = (int(i[0] - 0.7071*i[2]), int(i[1] - 0.7071*i[2]))
            pt3, pt4 = (int(i[0] + 0.7071*i[2]), int(i[1] + 0.7071*i[2]))
            cv2.rectangle(dst, (pt1, pt2), (pt3, pt4), (255, 255, 255), 3)
            #print(src_rgb[pt1:pt3, pt2:pt4, 0])
            #plt.imshow(src_rgb[pt2:pt4, pt1:pt3, :])
            #plt.show()
            target = src_hsv[pt2:pt4, pt1:pt3, :]
            mh, ms = np.median(target[:,:,0]), np.median(target[:,:,1])
            print((mh, ms))
            
            amh.append(mh)
            ams.append(ms)
            
        if len(amh) == 0:
            return None, (None, None), None
            
        else:
            return dst, (amh, ams), circles[0]
    
    def FillSample(self, size, show=False):
        print('SIZE', size)
        self.df = pd.DataFrame(columns=['Hue', 'Saturation', 'label'])
        circlecolor = ['Red', 'Green', 'Blue']
        detectH, detectS, label = [], [], []
        
        for clridx, cc in enumerate(circlecolor):
            cnt = 0
            for cci in range(50):
                if cnt > size:
                    break
                
                filename = 'D:/Circles/' + cc + str(cci) + '.jpg'
                
                if os.path.exists(filename):

                    print(filename)

                    dst, (amh, ams), circle = self.find_circle(filename=filename)
                    for mh, ms in zip(amh, ams):
                        detectH.append(mh)
                        detectS.append(ms)
                        label.append(clridx)

                    if show:   
                        plt.imshow(dst)
                    plt.show()

                    cnt = cnt + 1
                
        res = list(zip(detectH, detectS, label))

        self.df = pd.concat([self.df, pd.DataFrame(data=res, columns=['Hue', 'Saturation', 'label'])])
        print(self.df)
    
    def FillRandom(self, size):
        self.df = pd.DataFrame(columns=['Hue', 'Saturation', 'label'])
        
        data = []
        
        h1, s1 = random.sample(range(50, 100), size), random.sample(range(150, 200), size)
        data.extend([[h, s, 0] for h, s in zip(h1, s1)])

        h2, s2 = random.sample(range(150, 200), size), random.sample(range(150, 200), size)
        data.extend([[h, s, 1] for h, s in zip(h2, s2)])

        h3, s3 = random.sample(range(250, 300), size), random.sample(range(150, 200), size)
        data.extend([[h, s, 2] for h, s in zip(h3, s3)])
        
        self.df = pd.concat([self.df, pd.DataFrame(data=data, columns=['Hue', 'Saturation', 'label'])])
    
    def CreatePlot(self, show=False):        
        for i, marker in enumerate(self.markers):
            x = self.df[self.df['label']==i].iloc[:,0]
            y = self.df[self.df['label']==i].iloc[:,1]
            plt.scatter(x, y, marker=marker)
            
        plt.xlim([-360, 360])
        plt.ylim([0, 200])
        #plt.xlim([0, 360])
        #plt.ylim([0, 200])
        #plt.xlim([np.min(self.df.iloc[:,0])*0.8, np.max(self.df.iloc[:,0])*1.2])
        #plt.ylim([np.min(self.df.iloc[:,1])*0.8, np.max(self.df.iloc[:,1])*1.2])
        plt.savefig("plot.jpg", bbox_inches='tight', pad_inches=0, dpi=200)
        
        if show:
            plt.show()
        else:
            plt.close()
            
    def subclusterplot(self):
        df_ = self.X_train
        lab = self.kmeans.labels_
        df_['cluster'] = lab
        unique_clusters = np.unique(lab)
        unique_clusters = np.sort(np.setdiff1d(unique_clusters, [-1]))
        n_clusters = len(unique_clusters)
        
        centers = []
        for i in range(len(np.unique(df_['cluster']))):
            centers.append([np.mean(df_[df_['cluster'] == i]['Hue']), np.mean(df_[df_['cluster'] == i]['Saturation'])])
        
        for i in np.unique(df_['cluster']):
            x = df_[df_['cluster']==i].iloc[:,0]
            y = df_[df_['cluster']==i].iloc[:,1]
            plt.scatter(x, y, marker='s', edgecolor='k', s=50, label='Cluster' + str(i))

            center = centers[i]
            plt.scatter(center[0], center[1], s=200, color='white', alpha=0.8, edgecolor='k', marker='s')
            plt.scatter(center[0], center[1], s=70, color='k', edgecolor='k', marker='$%d$' % i)
        
        #plt.xlim([-360, 360])
        #plt.ylim([0, 200])
        plt.xlim([np.min(df_.iloc[:,0])*0.9-50, np.max(df_.iloc[:,0])*1.1+50])
        plt.ylim([np.min(df_.iloc[:,1])*0.9-10, np.max(df_.iloc[:,1])*1.1+10])
        plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.13),ncol=3, fancybox=True)
        
            
    def run(self, show=False):
        self.X_train = self.df.drop(['label'], axis=1)
        self.kmeans.fit(self.X_train)
        
        self.subclusterplot()
        plt.axis('off')
        plt.savefig("cluster.jpg", bbox_inches='tight', pad_inches=0, dpi=200)
        
        if show:
            plt.show()
        else:
            plt.close()
            
    def predict(self, img = None, filename = None, show=False):
        if img is None:
            dst, (amh, ams), circle = self.find_circle(filename=filename)
        elif filename is None:
            dst, (amh, ams), circle = self.find_circle(img=img)
            
        if dst is None:
            print('Found None')
            return None
        
        plt.imshow(dst)
        plt.axis('off')
        plt.savefig("entry_hough.jpg", bbox_inches='tight', pad_inches=0, dpi=200)
        plt.close()
        
        entry = pd.DataFrame(data=list(zip(amh,ams)), columns=['Hue', 'Saturation'])
        preds = self.kmeans.predict(entry)
        
        self.subclusterplot()
        plt.scatter(amh, ams, marker='*', color='r')
        plt.axis('off')
        plt.savefig("predict.jpg", bbox_inches='tight', pad_inches=0, dpi=200)
        
        if show:
            plt.show()
        else:
            plt.close()
            
        pred = np.argmax(np.bincount(preds))
        pos = circle[list(preds).index(pred)]
        return pred, pos
            
#master = RL()
#dst, (amh, ams) = master.find_circle(filename='D:Circles/red0.jpg')
#plt.imshow(dst)
#plt.axis('off')
#plt.savefig("entry_hough.jpg", bbox_inches='tight', pad_inches=0)
#plt.show()

#src = cv2.imread('C:/Users/kijun/entry_hough.jpg')
#src = cv2.cvtColor(src, cv2.COLOR_BGR2RGB)
#plt.imshow(src)
#plt.show()

#master.FillSample(10)
#master.CreatePlot(True)
#master.run(True)
#pred = master.predict('D:/Circles/red12.jpg', True)
#print(pred)

In [3]:
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from PyQt5.QtGui import QImage
from PyQt5 import QtGui
import serial
import time
import os

form_class = uic.loadUiType("RobotLab3.ui")[0]

class CamThread(QThread):
    finished = pyqtSignal(float)
    
    def __init__(self, cam):
        super().__init__()
        self.cam = cam
        self.power = True
        self.frame = None
        self.radius = None
        
        self.sample_size=10
        
    def run(self):
        video_capture = cv2.VideoCapture(1)
        while self.power:
            grabbed, self.frame = video_capture.read()
            frame = cv2.cvtColor(self.frame, cv2.COLOR_BGR2RGB)
            h, w, c = frame.shape
            
            if self.radius is not None:
                #Draw Circle
                cv2.circle(frame, (int(h/2) , int(w/2)), int(self.radius*min_ratio), (255,255,255), 2)
                cv2.circle(frame, (int(h/2) , int(w/2)), int(self.radius*max_ratio), (255,255,255), 2)
            
            qImg = QtGui.QImage(frame.data, w, h, w*c, QtGui.QImage.Format_RGB888)
            pixmap = QtGui.QPixmap.fromImage(qImg)
            self.cam.setPixmap(pixmap.scaled(420, 420, aspectRatioMode=Qt.KeepAspectRatio))
            
            
            self.cam.show()
    
    def stop(self):
        self.power = False
        self.quit()
        self.wait(1000)
        self.finished.emit(1)
        
class ClusterThread(QThread):
    finished = pyqtSignal(float)
    
    def __init__(self, ms, key, sample_size=10, filename=None, img=None):
        super().__init__()
        self.ms = ms
        self.key = key
        self.sample_size = sample_size
        self.pred = None
        self.pos = None
        self.filename = filename
        self.img = img
        
    def run(self):
        if self.key == 'learn':
            self.learn()
        elif self.key == 'predict':
            self.predict()
            
    def stop(self):
        self.quit()
        self.wait(500)
        self.finished.emit(0)
        
    def predict(self):
        if self.img is None:
            self.pred, self.pos = self.ms.predict(filename=self.filename)
            print(self.pred, self.pos)
        elif self.filename is None:
            self.pred, self.pos = self.ms.predict(img=self.img)
            print(self.pred, self.pos)
        
        self.finished.emit(1)
    
    def learn(self):
        self.ms.FillSample(self.sample_size)
        self.ms.CreatePlot()
        self.ms.run()
        
        self.finished.emit(1)
        #cluster_sample = 'C:/Users/kijun/cluster.jpg'
        #pixmap = QPixmap(cluster_sample)
        #self.Plot.setPixmap(pixmap.scaled(420,420,aspectRatioMode=Qt.KeepAspectRatio))

class BluetoothConnectThread(QThread):
    sig = pyqtSignal(str, object)
    
    def __init__(self):
        super().__init__()
        
    def run(self):
        try:
            bt = serial.Serial(port='COM14', baudrate=9600, timeout=1)
            self.sig.emit('Connected', bt)
        except:
            self.sig.emit('Not Connected', None)
        
class BluetoothThread(QThread):
    sig_word = pyqtSignal(str)
    sig_oper = pyqtSignal(str)
    work = ''
    msgs = []
    data = ''
    
    def __init__(self, bt):
        
        super().__init__()
        self.bluetooth = bt
        work = 'connect'
        msgs = []
        self.cluster = None
        self.pos = None
        
    def run(self):

        if self.work == 's':
            self.send()
        elif self.work == 'r':
            self.receive()
        elif self.work == 'sr':
            self.send_receive()
        elif self.work == 'm':
            self.main()
            
        return
    
    def set_send(self, msgs):
        self.work = 's'
        self.msgs = msgs
    def set_receive(self):
        self.work = 'r'
    def set_send_receive(self, msgs):
        self.work = 'sr'
        self.msgs = msgs
    def set_main(self):
        self.work = 'm'
        
    def main(self):
        self.sig_oper.emit('Waiting')
        while True:
            time.sleep(.1)
            
            self.receive()
            
            if self.data == '0':
                self.sig_oper.emit('Evaluate')
            
            if self.cluster is not None:
                #'#{}.', '!{}.', '*{}.'
                self.msgs = ['#{}.'.format(self.cluster), '!{}.'.format(self.pos[0]), '*{}.'.format(self.pos[1])]
                self.send()
                self.cluster = None
                
            #if self.cluster is None:
            #    self.msgs = ['#{}.'.format(5), '!{}.'.format(0), '*{}.'.format(0)]
            #    self.send()
            #    self.cluster = None
                
    def send(self):
        for msg in self.msgs:
            self.bluetooth.flushInput()
            time.sleep(.1)
            for ch in msg:
                self.bluetooth.write(bytes(ch, 'ascii'))
        
    def receive(self):
        self.bluetooth.flushOutput()
        time.sleep(.1)
        while True:
            if self.bluetooth.readable():
                self.data = self.bluetooth.readline().decode('ascii')
                if self.data != '':
                    self.sig_word.emit(str(self.data))
                return self.data
            
                
    def send_receive(self):
        for msg in self.msgs:
            self.bluetooth.flushInput()
            time.sleep(.1)
            for ch in msg:
                self.bluetooth.write(bytes(ch, 'ascii'))
                self.receive();
    
    
    def stop(self):
        self.quit()
        self.wait(500)
        self.finished.emit('Bluetooth Thread Done') 
        
        
class WindowClass(QMainWindow, form_class):
    def __init__(self):
        super().__init__()
        self.setupUi(self)
        
        self.ms = RL()

        # 이미지 파일 경로 목록을 저장할 리스트
        self.imageFilePaths = []

        # 버튼 클릭 이벤트 연결
        self.pushLearnCluster.clicked.connect(self.fGetSample)
        self.pushFromFile.clicked.connect(self.fFilePredict)
        self.pushCaptureCamera.clicked.connect(self.fCamPredict)
        self.pushCamOn.clicked.connect(self.fCamOn)
        self.pushCamOff.clicked.connect(self.fCamOff)
        self.pushSendBT.clicked.connect(self.fSendBTLineEdit)
        self.pushConnectBT.clicked.connect(self.fConnectBT)
        self.pushCreateDataWithCam.clicked.connect(self.fCaptureData)
        self.lineEditGuide.returnPressed.connect(self.fChangeRadius)
        self.checkBoxGuide.clicked.connect(self.fGuideEnable)
        self.pushDemo.clicked.connect(self.fRunSystemDemo)
        
        #camera
        self.video_thread = None
        self.learn_thread = None
        self.predict_thread = None
        self.bluetooth = None
        self.bluetooth_thread = None
        
        #self.bluetooth_thread = BluetoothThread('COM9')
        #self.bluetooth_thread.finished.connect(self.fBluetoothSerial)
        #self.bluetooth_thread.start()
        #self.bluetooth = serial.Serial(port='COM9', baudrate=9600,
        #                         parity=serial.PARITY_NONE,
        #                         stopbits=serial.STOPBITS_ONE,
        #                         bytesize=serial.EIGHTBITS)
        
        #learn
        #self.fGetSample()
        
    def fRunSystemDemo(self):
        self.log('sys', 'initiating')
        self.fGetSample()

        if self.bluetooth is None:
            self.log('sys', 'BT not connected')
            return
        
        self.bt_thread = BluetoothThread(self.bluetooth)
        self.bt_thread.sig_word.connect(self.fPrint_bt_msg)
        self.bt_thread.sig_oper.connect(self.f_oper)
        self.bt_thread.set_main()
        self.bt_thread.start()
    
    @pyqtSlot(str)
    def f_oper(self, oper):
        if oper == 'Waiting':
            self.log('sys', 'waiting')
        if oper == 'Evaluate':
            #self.fFilePredict()
            self.log('sys', 'predicting')
            self.fCamPredict()
    
    def fGuideEnable(self):
        self.fChangeRadius()
        
        #if self.video_thread is not None:
        #    if self.checkBoxGuide.isChecked():
        #        self.video_thread.radius = int(self.ms.radius)
        #    else:
        #        self.video_thread.radius = None
    
    def fChangeRadius(self):
        self.ms.radius = int(self.lineEditGuide.text())
        
        if self.video_thread is not None:
            if self.checkBoxGuide.isChecked():
                self.video_thread.radius = int(self.ms.radius)
            else:
                self.video_thread.radius = None
    
    def fCaptureData(self):
        if self.video_thread.frame is None:
            self.log('Create', 'No Cam')
            return
        key = self.comboBoxColor.currentText()
        #self.log('combo', key)
        
        for cci in range(50):
            filename = 'D:/Circles/' + key + str(cci) + '.jpg'
            if os.path.exists(filename):
                continue
            
            frame = cv2.cvtColor(self.video_thread.frame, cv2.COLOR_BGR2RGB)
            plt.imshow(frame)
            plt.axis('off')
            plt.savefig(filename, bbox_inches='tight', pad_inches=0, dpi=200)
            plt.close()
            self.log('Created', filename)
            return
        
        self.log('Create', 'failed')
        
    def fConnectBT(self):
        self.log('BT', 'Connecting...')
        self.bt_connect_thread = BluetoothConnectThread()
        self.bt_connect_thread.sig.connect(self.f_bt_connect_result)
        self.bt_connect_thread.start()
        #self.bluetooth_thread.connect()
    
    @pyqtSlot(str, object)
    def f_bt_connect_result(self, msg, bt):
        self.bluetooth = bt
        self.log('BT', msg)
    
    @pyqtSlot(str)
    def fPrint_bt_msg(self, msg):
        self.log('BT', msg)

        
    def fSendBTLineEdit(self):
        msg = self.lineEditBT.text()
        
        self.bt_thread = BluetoothThread(self.bluetooth)
        self.bt_thread.sig.connect(self.fPrint_bt_msg)
        self.bt_thread.set_send_receive(['#1.', '!{}.'.format(msg), '*321.'])
        self.bt_thread.start()
        
        #self.bluetooth_thread.send(msg)
        #self.bluetooth_thread.receive()
        #self.bluetooth_thread.send_receive(msg)
            #self.fSendBT(msg)
            
        #self.bluetooth_thread.work = 'send'
        #self.bluetooth_thread.msgs = ['#1.', '!{}.'.format(msg), '*321']
        #self.bluetooth_thread.send(['#1.', '!{}.'.format(msg), '*321'])
        #self.bluetooth_thread.start()
            
        #self.bluetooth_thread.send(['#1.', '!{}.'.format(msg), '*321'])
        
        #self.fSendBT('#1.')
        #self.fSendBT('!' + msg +'.')
        #self.fSendBT('*321.')
        #self.fReceiveBT()
    
    '''
    def fSendBT(self, msg):
        self.bluetooth.flushInput()
        time.sleep(.1)
        for ch in msg:
            self.bluetooth.write(bytes(ch, 'ascii'))
            self.fReceiveBT();
            #print('bt', ch)
        #self.bluetooth.write(bytes(msg, 'ascii'))
        #print('bt', msg)
        
        return
    
    def fSendBTChar(self, ch):
        self.bluetooth.flushInput()
        time.sleep(.1)
        self.bluetooth.write(bytes([ch]))
        
    def fReceiveBT(self):
        self.bluetooth.flushOutput()
        time.sleep(.1)
        while True:
            if self.bluetooth.readable():
                #data='Fake'
                data = self.bluetooth.readline().decode('ascii')
                self.log('BT', data)
                #print(data)
                return data
            
        
                #break
    '''
        
    def fBluetoothSerial(self, msg):
        self.listWidget.insertItem(0, msg)
        
    def fCamOn(self):
        if self.video_thread is None:
            self.video_thread = CamThread(self.Camera)
            self.video_thread.finished.connect(self.fDestroyCam)
            self.video_thread.start()
        
    def fCamOff(self):
        self.video_thread.stop()    
    
    @pyqtSlot(float)
    def fDestroyCam(self, val):
        self.video_thread = None
        self.Camera.clear()
        
        
    def fFilePredict(self):
        filename, _ = QFileDialog.getOpenFileName(self, "Open Image", "D:/Circles", "Image Files (*.png *.jpg *.jpeg *.bmp)")
        #filename = 'D:/Circles/red12.jpg'
        self.predict_thread = ClusterThread(self.ms, 'predict', filename=filename)
        self.predict_thread.finished.connect(self.fPredictDone)
        self.predict_thread.start()
        
        #pred = self.ms.predict(filename=filename)
        #print(pred)
        
        #cluster_sample = 'C:/Users/kijun/predict.jpg'
        #pixmap = QPixmap(cluster_sample)
        #self.Plot.setPixmap(pixmap.scaled(420,420,aspectRatioMode=Qt.KeepAspectRatio))
        
        #self.label_predict.setText(str(pred))
        
        #cluster_sample = 'C:/Users/kijun/entry_hough.jpg'
        #pixmap = QPixmap(cluster_sample)
        #self.Image.setPixmap(pixmap.scaled(420,420,aspectRatioMode=Qt.KeepAspectRatio))
        
    def fCamPredict(self):
        if self.video_thread.frame is None:
            return
        frame = cv2.cvtColor(self.video_thread.frame, cv2.COLOR_BGR2RGB)
        h, w, c = frame.shape
        qImg = QtGui.QImage(frame.data, w, h, w*c, QtGui.QImage.Format_RGB888)
        
        pixmap = QtGui.QPixmap.fromImage(qImg)
        self.Image.setPixmap(pixmap.scaled(420,420,aspectRatioMode=Qt.KeepAspectRatio))
        
        self.predict_thread = ClusterThread(self.ms, 'predict', img=self.video_thread.frame)
        self.predict_thread.finished.connect(self.fPredictDone)
        self.predict_thread.start()
        
    @pyqtSlot(float)    
    def fPredictDone(self, result):
        if result == 1: #Predict Done
            if self.predict_thread.pred is None: #Couldn't Estimate
                
                #self.SEND_DATA()
                
                self.log('Estimate', 'None')
                
                cluster_sample = 'C:/Users/kijun/cluster.jpg'
                pixmap = QPixmap(cluster_sample)
                self.Plot.setPixmap(pixmap.scaled(420,420,aspectRatioMode=Qt.KeepAspectRatio))
                
                
            else:    
                
                cluster_sample = 'C:/Users/kijun/predict.jpg'
                pixmap = QPixmap(cluster_sample)
                self.Plot.setPixmap(pixmap.scaled(420,420,aspectRatioMode=Qt.KeepAspectRatio))

                self.log('Estimate', str(self.predict_thread.pred))

                cluster_sample = 'C:/Users/kijun/entry_hough.jpg'
                pixmap = QPixmap(cluster_sample)
                self.Image.setPixmap(pixmap.scaled(420,420,aspectRatioMode=Qt.KeepAspectRatio))
                
                self.bt_thread.cluster = str(self.predict_thread.pred)
                self.bt_thread.pos = list(map(int, self.predict_thread.pos[:2]))
            
            self.predict_thread.stop()
            
        elif result == 0: #Destory Thread
            self.predict_thread = None
    
    def SEND_DATA(self):
        x, y = map(int, self.predict_thread.pos[:2])
        if self.predict_thread.pred is None:
            self.fSendBT('#{}.'.format('0'))
            self.fSendBT('!{}.'.format(str(x)))
            self.fSendBT('*{}.'.format(str(y)))
        elif self.predict_thread.pred == self.ms.red_class:
            self.fSendBT('#{}.'.format('1'))
            self.fSendBT('!{}.'.format(str(x)))
            self.fSendBT('*{}.'.format(str(y)))
        elif self.predict_thread.pred == self.ms.green_class:
            self.fSendBT('#{}.'.format('2'))
            self.fSendBT('!{}.'.format(str(x)))
            self.fSendBT('*{}.'.format(str(y)))
        else:
            self.fSendBT('#{}.'.format('3'))
            self.fSendBT('!{}.'.format(str(x)))
            self.fSendBT('*{}.'.format(str(y)))
        
        
        '''
        x, y = map(int, self.predict_thread.pos[:2])

        if self.predict_thread.pred is None:
            msg = str(0) + ',' + str(x) +',' + str(y) + '.'
        elif self.predict_thread.pred == self.ms.red_class:
            msg = str(1) + ',' + str(x) +',' + str(y) + '.'
        elif self.predict_thread.pred == self.ms.green_class:
            msg = str(2) + ',' + str(x) +',' + str(y) + '.'
        else:
            msg = str(3) + ',' + str(x) +',' + str(y) + '.'
        
        self.log('bt msg', msg)
        if self.bluetooth is not None: #Bluetooth is connected
            self.fSendBT(msg)
        '''
    
    def fGetSample(self):
        self.learn_thread = ClusterThread(self.ms, 'learn', sample_size = 5)
        self.learn_thread.finished.connect(self.fGetSampleDone)
        self.learn_thread.start()
        self.log('Cluster', 'Learning')
        
    def fGetSampleDone(self, result):
        self.ms = self.learn_thread.ms
        
        cluster_sample = 'C:/Users/kijun/cluster.jpg'
        pixmap = QPixmap(cluster_sample)
        self.Plot.setPixmap(pixmap.scaled(420,420,aspectRatioMode=Qt.KeepAspectRatio))
        self.log('Cluster', 'Done')
    
    def fCreateRandomSample(self):
        self.ms.FillRandom(20)
        self.ms.CreatePlot()
        self.ms.run()

        cluster_sample = 'C:/Users/kijun/cluster.jpg'
        pixmap = QPixmap(cluster_sample)
        self.Plot.setPixmap(pixmap.scaled(420,420,aspectRatioMode=Qt.KeepAspectRatio))
        
    def log(self, owner, msg):
        self.listWidget.insertItem(0, owner + ': ' + msg)
        

In [4]:
if __name__ == "__main__":
    app = QApplication(sys.argv)
    myWindow = WindowClass()
    myWindow.show()
    app.exec_()

SIZE 5
D:/Circles/Red0.jpg
(739, 985, 3)
(2.0, 219.0)
D:/Circles/Red1.jpg
(739, 985, 3)
(2.0, 207.0)
D:/Circles/Red2.jpg
(739, 985, 3)
(2.0, 216.5)
D:/Circles/Red3.jpg
(739, 985, 3)
(2.0, 217.0)
D:/Circles/Red4.jpg
(739, 985, 3)
(2.0, 211.0)
D:/Circles/Red5.jpg
(739, 985, 3)
(2.0, 224.0)
D:/Circles/Green0.jpg
(739, 985, 3)
(69.0, 121.0)
D:/Circles/Green1.jpg
(739, 985, 3)
(65.0, 119.0)
D:/Circles/Green2.jpg
(739, 985, 3)
(65.0, 127.0)
D:/Circles/Green3.jpg
(739, 985, 3)
(67.0, 122.5)
D:/Circles/Green4.jpg
(739, 985, 3)
(66.0, 120.0)
D:/Circles/Green5.jpg
(739, 985, 3)
(67.0, 133.0)
D:/Circles/Blue0.jpg
(739, 985, 3)
(116.0, 165.0)
D:/Circles/Blue1.jpg
(739, 985, 3)
(115.0, 161.5)
D:/Circles/Blue2.jpg
(739, 985, 3)
(116.0, 162.0)
D:/Circles/Blue3.jpg
(739, 985, 3)
(114.0, 161.0)
D:/Circles/Blue4.jpg
(739, 985, 3)
(116.0, 159.0)
D:/Circles/Blue5.jpg
(739, 985, 3)
(115.0, 148.0)
      Hue  Saturation label
0     2.0       219.0     0
1     2.0       207.0     0
2     2.0       216.5     0

C:\Users\kijun\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


(480, 640, 3)
Found None


TypeError: cannot unpack non-iterable NoneType object